In [36]:
import numpy as np
import pandas as pd
from matplotlib import pyplot
from base64 import b64decode as decode


In [67]:
# use class base64 to decode waveform data
def to_array(wf):
    barr = bytearray(decode(wf))
    vals = np.array(barr)
    return vals.view(np.int16)

# read in data
exam_data = pd.read_csv("data/d_exam.csv").drop(columns = ["site_num", "patient_id_edit"])
waveform_data = pd.read_csv("data/d_waveform.csv")
lead_data = pd.read_csv("data/d_lead_data.csv").drop(columns = ["exam_id"])
diagnosis_data = pd.read_csv("data/d_diagnosis.csv").drop(columns = ["user_input"])

# add decoded data as a column to lead data
waveforms = list(lead_data['waveform_data'])
lead_data['decoded_waveform'] = [to_array(i) for i in waveforms]



,waveform_id,exam_id,waveform_type,number_of_leads,Waveform_Start_Time,Sample_Type,Sample_Base,Sample_Exponent,High_Pass_Filter,Low_Pass_Filter,AC_Filter
0,1095618,549871,Rhythm,8,0,CONTINUOUS_SAMPLES,250,0,5,150,NONE
1,1097350,548759,Median,8,0,CONTINUOUS_SAMPLES,250,0,5,150,NONE
2,1098641,548759,Rhythm,8,0,CONTINUOUS_SAMPLES,250,0,5,150,NONE
3,1099412,549871,Median,8,0,CONTINUOUS_SAMPLES,250,0,5,150,NONE
4,1100359,551485,Rhythm,8,0,CONTINUOUS_SAMPLES,250,0,5,150,NONE


,lead_data_id,waveform_id,WavfmType,lead_id,lead_byte_count_total,lead_time_offset,waveform_data,lead_sample_count_total,lead_amplitude,lead_units,lead_high_limit,lead_low_limit,lead_offset_first_sample,first_sample_baseline,lead_sample_Size,lead_off,baseline_sway,lead_data_crc32,decoded_waveform
0,9066567,1098641,NaN,I,5000,0,BAADAAIA///8//z//P/8//z/+f/2//j/+v/6//r/+f/4/...,2500,4.88,MICROVOLTS,32767,-32768,0,0,2,False,False,3657741200,"[4, 3, 2, -1, -4, -4, -4, -4, -4, -7, -10, -8,..."
1,9066887,1095618,NaN,V2,5000,0,9v/1//T/9P/0//T/9P/0//T/9f/2//b/9v/2//b/9v/2/...,2500,4.88,MICROVOLTS,32767,-32768,0,0,2,False,False,3243295087,"[-10, -11, -12, -12, -12, -12, -12, -12, -12, ..."
2,9067934,1099412,NaN,V2,600,0,BwAIAAgACAAHAAYABgAGAAYABgAGAAYABgAGAAYABgAGA...,300,4.88,MICROVOLTS,32767,-32768,0,0,2,False,False,3257514197,"[7, 8, 8, 8, 7, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, ..."
3,9068730,1098641,NaN,V6,5000,0,BgAEAAIAAgACAAAA/v/+//7//f/8//z//P/8//z//P/8/...,2500,4.88,MICROVOLTS,32767,-32768,0,0,2,False,False,3316670153,"[6, 4, 2, 2, 2, 0, -2, -2, -2, -3, -4, -4, -4,..."
4,9069190,1098641,NaN,V5,5000,0,CAAFAAIAAQAAAP7//P/7//r/+v/6//n/+P/4//j/+P/4/...,2500,4.88,MICROVOLTS,32767,-32768,0,0,2,False,False,4234387713,"[8, 5, 2, 1, 0, -2, -4, -5, -6, -6, -6, -7, -8..."
